In [1]:
%pip install pydiffmap

In [2]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.manifold import Isomap, LocallyLinearEmbedding, TSNE
from pydiffmap import diffusion_map

In [3]:
from zipfile import ZipFile
file_name = '/content/Dataset.zip'

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


# Data Preprocessing

In [4]:
# Define the image dimensions
IMG_WIDTH, IMG_HEIGHT = 128, 128

# Define the path to the dataset
dataset_path = 'Dataset'

# Define a dictionary to map folder names to class labels
class_labels = {'Mild_Demented': 0, 'Moderate_Demented': 1, 'Non_Demented': 2, 'Very_Mild_Demented': 3}

# Function to load and preprocess images
def load_and_preprocess_images(dataset_path, img_width, img_height):
    images = []
    labels = []
    for class_name, class_label in class_labels.items():
        class_folder = os.path.join(dataset_path, class_name)
        for img_file in os.listdir(class_folder):
            img_path = os.path.join(class_folder, img_file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, (img_width, img_height))
                images.append(img.flatten())
                labels.append(class_label)
    return np.array(images), np.array(labels)

# Load and preprocess the images
X, y = load_and_preprocess_images(dataset_path, IMG_WIDTH, IMG_HEIGHT)

# Print the shape of the processed data
print(f'Processed data shape: {X.shape}')
print(f'Processed labels shape: {y.shape}')

Processed data shape: (6400, 16384)
Processed labels shape: (6400,)


# Apply the NLDR techniques

In [8]:
# Initialize ISOMAP with best parameters
isomap = Isomap(n_components = 44)

# Fit ISOMAP to the preprocessed data
X_isomap = isomap.fit_transform(X)

# Initialize LLE with best parameters
lle = LocallyLinearEmbedding(n_components = 1030, eigen_solver = 'dense')

# Fit LLE to the preprocessed data
X_lle = lle.fit_transform(X)

# Initialize Diffusion Map
neighbor_params = {'algorithm': 'brute'}
diffmap = diffusion_map.DiffusionMap.from_sklearn(n_evecs = 1033, neighbor_params = neighbor_params, alpha = 0.25)

# Fit Diffusion Map to the preprocessed data
X_diffmap = diffmap.fit_transform(X)

# Initialize t-SNE with best parameters for 2 dimensions
tsne2D = TSNE(n_components = 2, perplexity = 10, learning_rate = 200, init = 'random', random_state = 42)

# Fit t-SNE to the preprocessed data
X_tsne_2D = tsne2D.fit_transform(X)

# Initialize t-SNE with best parameters for 3 dimensions
tsne3D = TSNE(n_components = 3, perplexity = 10, learning_rate = 100, init = 'random', random_state = 42)

# Fit t-SNE to the preprocessed data
X_tsne_3D = tsne3D.fit_transform(X)